In [1]:
# Import our dependencies
import numpy as np
import pandas as pd
import polars as pl

In [2]:
#  Import and read the input csv
MDp10k_df = pd.read_csv('../../Raw_Data/WHO_Medical_Doctors_per10k-data.csv')
MDp10k_df

,IndicatorCode,Indicator,ValueType,ParentLocationCode,ParentLocation,Location type,SpatialDimValueCode,Location,Period type,Period,...,FactValueUoM,FactValueNumericLowPrefix,FactValueNumericLow,FactValueNumericHighPrefix,FactValueNumericHigh,Value,FactValueTranslationID,FactComments,Language,DateModified
0,HWF_0001,"Medical doctors (per 10,000)",numeric,AFR,Africa,Country,TCD,Chad,Year,2021,...,NaN,NaN,NaN,NaN,NaN,0.58,NaN,NaN,EN,2023-01-12T05:00:00.000Z
1,HWF_0001,"Medical doctors (per 10,000)",numeric,AFR,Africa,Country,TGO,Togo,Year,2021,...,NaN,NaN,NaN,NaN,NaN,0.59,NaN,NaN,EN,2023-01-12T05:00:00.000Z
2,HWF_0001,"Medical doctors (per 10,000)",numeric,WPR,Western Pacific,Country,PNG,Papua New Guinea,Year,2021,...,NaN,NaN,NaN,NaN,NaN,0.63,NaN,NaN,EN,2023-01-12T05:00:00.000Z
3,HWF_0001,"Medical doctors (per 10,000)",numeric,AFR,Africa,Country,BDI,Burundi,Year,2021,...,NaN,NaN,NaN,NaN,NaN,0.65,NaN,NaN,EN,2023-01-12T05:00:00.000Z
4,HWF_0001,"Medical doctors (per 10,000)",numeric,AFR,Africa,Country,MOZ,Mozambique,Year,2021,...,NaN,NaN,NaN,NaN,NaN,0.81,NaN,NaN,EN,2023-01-12T05:00:00.000Z
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2898,HWF_0001,"Medical doctors (per 10,000)",numeric,EMR,Eastern Mediterranean,Country,OMN,Oman,Year,1990,...,NaN,NaN,NaN,NaN,NaN,7.99,NaN,NaN,EN,2023-01-12T05:00:00.000Z
2899,HWF_0001,"Medical doctors (per 10,000)",numeric,AMR,Americas,Country,COL,Colombia,Year,1990,...,NaN,NaN,NaN,NaN,NaN,8.03,NaN,NaN,EN,2023-01-12T05:00:00.000Z
2900,HWF_0001,"Medical doctors (per 10,000)",numeric,WPR,Western Pacific,Country,KOR,Republic of Korea,Year,1990,...,NaN,NaN,NaN,NaN,NaN,8.11,NaN,NaN,EN,2023-01-12T05:00:00.000Z
2901,HWF_0001,"Medical doctors (per 10,000)",numeric,EUR,Europe,Country,TUR,Türkiye,Year,1990,...,NaN,NaN,NaN,NaN,NaN,9.32,NaN,NaN,EN,2023-01-12T05:00:00.000Z


In [3]:
# Drop the population estimate column
MDp10k_df = MDp10k_df.drop(['IndicatorCode','ValueType','ParentLocationCode','ParentLocation','Location type','SpatialDimValueCode','Period type'], axis=1)
MDp10k_df = MDp10k_df.drop(['FactValueUoM','FactValueNumericLowPrefix','FactValueNumericLow','FactValueNumericHighPrefix','FactValueNumericHigh'], axis=1)
MDp10k_df = MDp10k_df.drop(['FactValueTranslationID','FactComments','Language','DateModified','IsLatestYear','Dim1 type','Dim1','Dim2','Dim3',], axis=1)
MDp10k_df = MDp10k_df.drop(['Dim1ValueCode','Dim2ValueCode','Dim3ValueCode','Dim2 type','Dim3 type','DataSource','FactValueNumericPrefix'], axis=1)
MDp10k_df = MDp10k_df.drop(['FactValueNumeric','DataSourceDimValueCode','Indicator'], axis=1)
MDp10k_df.rename(columns ={'Location':'Country', 'Period':'Year', 'Value':'MD per 10k'}, inplace = True)
MDp10k_df

,Country,Year,MD per 10k
0,Chad,2021,0.58
1,Togo,2021,0.59
2,Papua New Guinea,2021,0.63
3,Burundi,2021,0.65
4,Mozambique,2021,0.81
...,...,...,...
2898,Oman,1990,7.99
2899,Colombia,1990,8.03
2900,Republic of Korea,1990,8.11
2901,Türkiye,1990,9.32


In [4]:
# Open the list of country name corrections
corrections_df = pd.read_csv('../../Clean_Data/master_country_list/country_name_corrections.csv')

# Convert the corrections dataframe to a dictionary.
correction_dict = dict(zip(corrections_df['wrong'], corrections_df['correct']))

In [5]:
# Apply the correction dictionary to fix the known errors
MDp10k_df['Country'] = MDp10k_df['Country'].replace(correction_dict)

In [6]:
# Open the master list of countries
countries_df = pd.read_csv('../../Clean_Data/master_country_list/country_profile_urls.csv')
countries_df = countries_df.drop(['profile_url'], axis=1)

In [7]:
country_list = countries_df['country'].tolist()
country_list

['Albania',
 'Algeria',
 'Andorra',
 'Angola',
 'Antigua and Barbuda',
 'Argentina',
 'Armenia',
 'Aruba',
 'Australia',
 'Austria',
 'Azerbaijan',
 'Bahamas',
 'Bahrain',
 'Bangladesh',
 'Barbados',
 'Belarus',
 'Belgium',
 'Belize',
 'Benin',
 'Bermuda',
 'Bhutan',
 'Bolivia',
 'Botswana',
 'Brazil',
 'British Virgin Islands',
 'Brunei',
 'Bulgaria',
 'Burkina Faso',
 'Burundi',
 'Cabo Verde',
 'Cambodia',
 'Cameroon',
 'Canada',
 'Central African Republic',
 'Chad',
 'Chile',
 'China',
 'Colombia',
 'Congo',
 'Democratic Republic of the Congo',
 'Costa Rica',
 'Croatia',
 'Cuba',
 'Cyprus',
 'Czechia',
 "Côte d'Ivoire",
 'Denmark',
 'Djibouti',
 'Dominica',
 'Dominican Republic',
 'Ecuador',
 'Egypt',
 'El Salvador',
 'Equatorial Guinea',
 'Estonia',
 'Eswatini',
 'Ethiopia',
 'Fiji',
 'Finland',
 'France',
 'Gabon',
 'Gambia',
 'Georgia',
 'Germany',
 'Ghana',
 'Greece',
 'Grenada',
 'Guatemala',
 'Guinea',
 'Guinea-Bissau',
 'Guyana',
 'Haiti',
 'Honduras',
 'Hong Kong (China)',
 

In [8]:
# Create a list of MDp10k countries
MDp10k_countries = MDp10k_df['Country'].tolist()
MDp10k_countries

['Chad',
 'Togo',
 'Papua New Guinea',
 'Burundi',
 'Mozambique',
 'Cameroon',
 'Tonga',
 'Sri Lanka',
 'Peru',
 'Brunei',
 'Guinea-Bissau',
 'Kenya',
 'Zambia',
 'Brazil',
 'Colombia',
 'Canada',
 'Montenegro',
 'Costa Rica',
 'Saudi Arabia',
 'El Salvador',
 'Chile',
 'Laos',
 'Nigeria',
 'United Kingdom',
 'Paraguay',
 'Hungary',
 'Trinidad and Tobago',
 'New Zealand',
 'Israel',
 'Ireland',
 'Italy',
 'Switzerland',
 'Germany',
 'Slovakia',
 'Lithuania',
 'Bhutan',
 'Norway',
 'Cyprus',
 'Georgia',
 'Austria',
 'Czechia',
 'Malta',
 'Bangladesh',
 'Indonesia',
 'Uruguay',
 'Belgium',
 'Philippines',
 'South Africa',
 'Nepal',
 'Niger',
 'Malawi',
 'Chad',
 'Burundi',
 'Sierra Leone',
 'Gambia',
 'Togo',
 'Eritrea',
 'Mozambique',
 'Senegal',
 'Cameroon',
 'Ethiopia',
 'Eswatini',
 'Uganda',
 'Ghana',
 'Zimbabwe',
 'Guinea-Bissau',
 'Sri Lanka',
 'Tuvalu',
 'Guatemala',
 'Guyana',
 'Panama',
 'Palau',
 'Albania',
 'Oman',
 'Afghanistan',
 'Zambia',
 'Türkiye',
 'Malaysia',
 'Kuwait'

In [9]:
# clean the FS data based on the SS country list
no_match = []

for country in MDp10k_countries:
    if country in country_list:
        continue
    else:
        no_match.append(country)

In [10]:
MDp10k_clean_df = MDp10k_df.drop(MDp10k_df[MDp10k_df['Country'].isin(no_match)].index.tolist())
MDp10k_clean_df

,Country,Year,MD per 10k
0,Chad,2021,0.58
1,Togo,2021,0.59
2,Papua New Guinea,2021,0.63
3,Burundi,2021,0.65
4,Mozambique,2021,0.81
...,...,...,...
2898,Oman,1990,7.99
2899,Colombia,1990,8.03
2900,South Korea,1990,8.11
2901,Türkiye,1990,9.32


In [11]:
MDp10k_sorted_df = MDp10k_clean_df.sort_values(['Country','Year'], axis=0)

In [12]:
MDp10k_sorted_df.tail(40)

,Country,Year,MD per 10k
1626,Vietnam,2007,6.47
1519,Vietnam,2008,6.70
1395,Vietnam,2009,7.03
1272,Vietnam,2010,7.15
1038,Vietnam,2012,8.26
810,Vietnam,2014,7.87
701,Vietnam,2015,8.01
593,Vietnam,2016,8.33
2459,Yemen,1997,2.23
1915,Yemen,2004,3.25


In [13]:
# Create a complete dataframe with all countries and all the years.

# Define variables to collect the details.
#data_countries = MDp10k_sorted_df['Country'].unique()
data_countries = country_list
years = list(range(1990, 2023))

# Create a helper DataFrame.
helper_df = pd.DataFrame({'Country':np.repeat(data_countries, len(years)), 'Year':np.tile(years, len(data_countries))})

# Merge the helper DataFrame with the original data to ensure the data range is complete.
complete_df = pd.merge(helper_df, MDp10k_sorted_df, on=['Country','Year'], how='left')

In [14]:
# Convert to wide format
wide_df = complete_df.pivot(index='Year', columns='Country', values='MD per 10k')

# Flatten the hierarchical columns and create 'Country_Series' style column names
wide_df.columns = [f'{col}_MD per 10k' for col in wide_df.columns]

# Reset the index, so 'Year' becomes a column again
wide_df.reset_index(inplace=True)

# Convert columns to numeric, coerce non-numeric values to NaN
for col in wide_df.columns[0:]:
    wide_df[col] = pd.to_numeric(wide_df[col], errors='coerce')

# Convert to Polars DataFrame
wide_pl_df = pl.from_pandas(wide_df)
    
wide_pl_df.head(5)

Year,Albania_MD per 10k,Algeria_MD per 10k,Andorra_MD per 10k,Angola_MD per 10k,Antigua and Barbuda_MD per 10k,Argentina_MD per 10k,Armenia_MD per 10k,Aruba_MD per 10k,Australia_MD per 10k,Austria_MD per 10k,Azerbaijan_MD per 10k,Bahamas_MD per 10k,Bahrain_MD per 10k,Bangladesh_MD per 10k,Barbados_MD per 10k,Belarus_MD per 10k,Belgium_MD per 10k,Belize_MD per 10k,Benin_MD per 10k,Bermuda_MD per 10k,Bhutan_MD per 10k,Bolivia_MD per 10k,Botswana_MD per 10k,Brazil_MD per 10k,British Virgin Islands_MD per 10k,Brunei_MD per 10k,Bulgaria_MD per 10k,Burkina Faso_MD per 10k,Burundi_MD per 10k,Cabo Verde_MD per 10k,Cambodia_MD per 10k,Cameroon_MD per 10k,Canada_MD per 10k,Central African Republic_MD per 10k,Chad_MD per 10k,Chile_MD per 10k,…,Serbia_MD per 10k,Seychelles_MD per 10k,Sierra Leone_MD per 10k,Singapore_MD per 10k,Slovakia_MD per 10k,Slovenia_MD per 10k,Solomon Islands_MD per 10k,South Africa_MD per 10k,South Korea_MD per 10k,Spain_MD per 10k,Sri Lanka_MD per 10k,Sudan_MD per 10k,Suriname_MD per 10k,Sweden_MD per 10k,Switzerland_MD per 10k,Syria_MD per 10k,Taiwan (China)_MD per 10k,Tajikistan_MD per 10k,Tanzania_MD per 10k,Thailand_MD per 10k,Togo_MD per 10k,Trinidad and Tobago_MD per 10k,Tunisia_MD per 10k,Turkmenistan_MD per 10k,Türkiye_MD per 10k,Uganda_MD per 10k,Ukraine_MD per 10k,United Kingdom_MD per 10k,United States_MD per 10k,Uruguay_MD per 10k,Uzbekistan_MD per 10k,Vanuatu_MD per 10k,Venezuela_MD per 10k,Vietnam_MD per 10k,Yemen_MD per 10k,Zambia_MD per 10k,Zimbabwe_MD per 10k
i32,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,…,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64
1990,13.71,null,null,null,null,null,null,null,21.7,30.09,37.01,null,null,null,null,null,32.68,null,null,null,null,null,null,10.74,null,null,32.5,null,null,null,null,null,null,null,null,null,…,null,null,null,null,null,20.7,null,null,8.11,20.47,null,null,null,32.53,29.84,null,null,24.97,null,null,null,null,5.24,null,9.32,null,null,16.18,null,null,33.61,null,null,null,null,null,1.31
1991,14.52,null,null,null,null,null,null,null,23.36,30.88,37.32,null,null,null,null,null,33.31,null,null,null,null,null,null,null,null,null,30.68,null,null,null,null,null,null,null,null,null,…,null,null,null,null,null,20.88,null,null,8.75,39.31,1.4,null,null,33.7,30.31,null,null,25.49,null,2.28,null,null,5.22,null,9.85,null,null,16.33,null,null,33.5,null,null,null,null,null,null
1992,15.93,null,null,null,null,26.45,null,null,23.74,32.13,36.87,null,null,null,null,null,34.01,null,null,null,null,null,null,11.28,null,null,31.19,null,null,null,null,null,null,null,null,9.9,…,null,null,null,null,null,20.94,null,null,9.48,39.82,1.67,null,null,34.79,30.13,null,null,23.11,null,null,null,null,5.79,null,10.12,null,null,16.46,null,null,33.41,null,null,null,null,null,null
1993,13.68,null,null,null,null,null,null,null,23.99,32.96,37.24,null,null,null,null,null,34.59,null,null,null,null,null,null,11.55,null,null,33.05,null,null,null,null,null,null,null,null,null,…,null,null,null,null,null,20.28,null,null,10.12,40.41,2.08,null,null,35.8,30.62,null,null,21.22,null,2.36,null,null,5.86,null,10.66,null,null,16.84,null,null,33.11,null,null,null,null,null,null
1994,12.85,null,null,null,null,null,null,null,24.38,34.08,36.84,null,null,null,null,null,35.24,null,null,null,null,null,null,11.84,null,null,32.86,null,null,null,null,null,null,null,null,null,…,null,null,null,null,null,21.87,null,null,10.57,40.91,1.86,null,null,36.8,31.16,null,null,21.78,null,null,null,null,5.85,null,11.29,null,null,16.98,null,null,32.81,null,null,null,null,null,null


In [15]:
# Handle the missing data

wide_pl_df = wide_pl_df.interpolate()
wide_pl_df = wide_pl_df.fill_null(strategy='backward')
wide_pl_df = wide_pl_df.fill_null(strategy='forward')
wide_pl_df = wide_pl_df.fill_null(strategy='zero')

In [16]:
wide_pl_df.head(15)

Year,Albania_MD per 10k,Algeria_MD per 10k,Andorra_MD per 10k,Angola_MD per 10k,Antigua and Barbuda_MD per 10k,Argentina_MD per 10k,Armenia_MD per 10k,Aruba_MD per 10k,Australia_MD per 10k,Austria_MD per 10k,Azerbaijan_MD per 10k,Bahamas_MD per 10k,Bahrain_MD per 10k,Bangladesh_MD per 10k,Barbados_MD per 10k,Belarus_MD per 10k,Belgium_MD per 10k,Belize_MD per 10k,Benin_MD per 10k,Bermuda_MD per 10k,Bhutan_MD per 10k,Bolivia_MD per 10k,Botswana_MD per 10k,Brazil_MD per 10k,British Virgin Islands_MD per 10k,Brunei_MD per 10k,Bulgaria_MD per 10k,Burkina Faso_MD per 10k,Burundi_MD per 10k,Cabo Verde_MD per 10k,Cambodia_MD per 10k,Cameroon_MD per 10k,Canada_MD per 10k,Central African Republic_MD per 10k,Chad_MD per 10k,Chile_MD per 10k,…,Serbia_MD per 10k,Seychelles_MD per 10k,Sierra Leone_MD per 10k,Singapore_MD per 10k,Slovakia_MD per 10k,Slovenia_MD per 10k,Solomon Islands_MD per 10k,South Africa_MD per 10k,South Korea_MD per 10k,Spain_MD per 10k,Sri Lanka_MD per 10k,Sudan_MD per 10k,Suriname_MD per 10k,Sweden_MD per 10k,Switzerland_MD per 10k,Syria_MD per 10k,Taiwan (China)_MD per 10k,Tajikistan_MD per 10k,Tanzania_MD per 10k,Thailand_MD per 10k,Togo_MD per 10k,Trinidad and Tobago_MD per 10k,Tunisia_MD per 10k,Turkmenistan_MD per 10k,Türkiye_MD per 10k,Uganda_MD per 10k,Ukraine_MD per 10k,United Kingdom_MD per 10k,United States_MD per 10k,Uruguay_MD per 10k,Uzbekistan_MD per 10k,Vanuatu_MD per 10k,Venezuela_MD per 10k,Vietnam_MD per 10k,Yemen_MD per 10k,Zambia_MD per 10k,Zimbabwe_MD per 10k
i32,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,…,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64
1990,13.71,11.18,22.73,0.59,1.63,26.45,26.16,0.0,21.7,30.09,37.01,9.89,9.53,2.47,13.65,46.18,32.68,10.44,0.39,0.0,1.82,11.81,2.56,10.74,0.0,10.06,32.5,0.53,0.28,4.87,1.12,1.86,24.28,0.8,0.26,9.9,…,26.05,14.29,0.3,13.43,32.4,20.7,1.29,7.17,8.11,20.47,1.4,2.62,3.99,32.53,29.84,13.03,0.0,24.97,0.23,2.28,0.44,9.05,5.24,27.16,9.32,0.81,30.27,16.18,24.34,37.46,33.61,1.12,19.29,5.3,2.23,1.34,1.31
1991,14.52,11.18,22.73,0.59,1.63,26.45,26.16,0.0,23.36,30.88,37.32,9.89,9.53,2.47,13.65,46.18,33.31,10.44,0.39,0.0,1.82,11.81,2.56,11.01,0.0,10.06,30.68,0.53,0.28,4.87,1.12,1.86,24.28,0.8,0.26,9.9,…,26.05,14.29,0.3,13.43,32.4,20.88,1.29,7.17,8.75,39.31,1.4,2.62,3.99,33.7,30.31,13.03,0.0,25.49,0.23,2.28,0.44,9.05,5.22,27.16,9.85,0.81,30.27,16.33,24.34,37.46,33.5,1.12,19.29,5.3,2.23,1.34,1.344
1992,15.93,11.18,22.73,0.59,1.63,26.45,26.16,0.0,23.74,32.13,36.87,9.89,9.53,2.47,13.65,46.18,34.01,10.44,0.39,0.0,1.82,11.81,2.56,11.28,0.0,10.06,31.19,0.53,0.28,4.87,1.12,1.86,24.28,0.8,0.26,9.9,…,26.05,14.29,0.3,13.43,32.4,20.94,1.29,7.17,9.48,39.82,1.67,2.62,3.99,34.79,30.13,13.03,0.0,23.11,0.23,2.32,0.44,9.05,5.79,27.16,10.12,0.81,30.27,16.46,24.34,37.46,33.41,1.12,19.29,5.3,2.23,1.34,1.378
1993,13.68,11.18,22.73,0.59,1.63,27.046667,26.16,0.0,23.99,32.96,37.24,9.89,9.53,2.47,13.65,46.18,34.59,10.44,0.39,0.0,1.82,11.81,2.56,11.55,0.0,10.06,33.05,0.53,0.28,4.87,1.12,1.86,24.28,0.8,0.26,9.802,…,26.05,14.29,0.3,13.43,32.4,20.28,1.29,7.17,10.12,40.41,2.08,2.62,3.99,35.8,30.62,13.03,0.0,21.22,0.23,2.36,0.44,9.05,5.86,27.16,10.66,0.81,30.27,16.84,24.34,37.46,33.11,1.12,19.29,5.3,2.23,1.34,1.412
1994,12.85,11.18,22.73,0.59,1.63,27.643333,26.16,0.0,24.38,34.08,36.84,9.89,9.53,2.47,13.65,46.18,35.24,10.44,0.39,0.0,1.82,11.81,2.56,11.84,0.0,10.06,32.86,0.53,0.28,4.87,1.12,1.86,24.28,0.8,0.26,9.704,…,26.05,14.29,0.3,13.43,32.4,21.87,1.29,7.17,10.57,40.91,1.86,2.62,3.99,36.8,31.16,13.03,0.0,21.78,0.23,2.375,0.44,9.05,5.85,27.16,11.29,0.81,30.27,16.98,24.34,37.46,32.81,1.12,19.29,5.3,2.23,1.34,1.446
1995,12.92,11.18,22.73,0.59,1.63,28.24,26.16,0.0,24.76,35.12,37.03,9.89,9.53,2.47,13.65,46.18,35.58,10.44,0.39,0.0,1.82,11.81,2.56,12.14,0.0,10.06,34.29,0.53,0.28,4.87,1.12,1.86,24.28,0.8,0

In [17]:
# Reformat the dataframe with the filled values...
# Convert the DataFrame back to pandas!!!
wide_df = wide_pl_df.to_pandas() 

# Now we can reshape it with pandas functionality
complete_df = wide_df.melt(id_vars='Year', value_vars=wide_df.columns[1:], var_name='Country', value_name='MD per 10k')

# Removing "_MD per 10k" from country names
complete_df['Country'] = complete_df['Country'].str.replace('_MD per 10k', '')

# Creating 'Country_Year' column
complete_df['Country_Year'] = complete_df['Country'] + '_' + complete_df['Year'].astype(str)

complete_df = complete_df.loc[:, ['Country_Year', 'Country', 'Year', 'MD per 10k']]

In [18]:
complete_df

,Country_Year,Country,Year,MD per 10k
0,Albania_1990,Albania,1990,13.71
1,Albania_1991,Albania,1991,14.52
2,Albania_1992,Albania,1992,15.93
3,Albania_1993,Albania,1993,13.68
4,Albania_1994,Albania,1994,12.85
...,...,...,...,...
6100,Zimbabwe_2018,Zimbabwe,2018,2.01
6101,Zimbabwe_2019,Zimbabwe,2019,1.95
6102,Zimbabwe_2020,Zimbabwe,2020,1.89
6103,Zimbabwe_2021,Zimbabwe,2021,1.89


In [20]:
complete_df.to_csv('../../Clean_Data/Clean_CSV_Files/MD_per_10k.csv', index=False)